# 第三节 探索用户行为模式


## 1. 项目背景


* 网购已经成为人们生活不可或缺的一部分，本次项目基于淘宝app平台数据,
通过相关指标对用户行为进行分析,从而探索用户相关行为模式。



* 提出问题

1.日PV有多少

2.日UV有多少

3.付费率情况如何

4.复购率是多少

5.漏斗流失情况如何

6.用户价值情况

列字段分别是
* user_id：用户身份，脱敏

* item_id：商品ID，脱敏

* behavior_type：用户行为类型（包含点击、收藏、加购物车、支付四种行为，分别用数字1、2、3、4表示）

* user_geohash：地理位置

* item_category：品类ID（商品所属的品类）

* time：用户行为发生的时间

## 2. 理解数据


In [ ]:
#导入相关库
import pandas as pd
import numpy as py
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
sns.set(style='darkgrid',context='notebook',font_scale=1.5) # 设置背景
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False #处理中文和坐标负号显示

In [ ]:
import os
os.chdir(r'E:\云开明培训机构\云开见明培训课件\data summary\第六章')

In [ ]:
data_user=pd.read_csv('tianchi_mobile_recommend_train_user.csv',dtype = str)

In [ ]:
data_user.info()

In [ ]:
data_user.head(5)

## 3. 数据预处理


### 3.1 缺失值处理


In [ ]:
#统计缺失值
data_user.apply(lambda x: sum(x.isnull()),axis=0)

* 存在缺失值的是User_geohash，有8334824条，不能删除缺失值，因为地理信息在数据集收集过程中做过加密转换，因此对数据集不做处理。

### 3.2 一致化处理

* 用户行为发生的时间有日期和小时构成，需要将该列进行拆分

In [ ]:
#拆分数据集
data_user['date']= data_user['time'].str[0:10].str.strip()
data_user['hour']= data_user['time'].str[11:].str.strip()
data_user.head()

In [ ]:
# 查看data_user数据集数据类型：
data_user.dtypes

In [ ]:
# time ,date应该为日期格式，hour应该为整数型
# 进行日期格式转换
#数据类型转化
data_user['date']=pd.to_datetime(data_user['date'])
data_user['time']=pd.to_datetime(data_user['time'])
data_user['hour']=data_user['hour'].astype('int64')

In [ ]:
data_user.dtypes#检查数据类型结果


### 3.3 异常值情况

In [ ]:
data_user.sort_values(by='time',ascending=True,inplace =True)#排序处理

In [ ]:
data_user.reset_index(drop=True,inplace =True)#建立索引

In [ ]:
data_user.describe(include = ['object'])

* 通过观察数据集的四分位数，总数，平均值，方差等，发现数据集并无异常值存在。

## 4. 构建模型(用户行为分析)

### 4.1 pv和uv分析

* PV(访问量)：即Page View, 具体是指网站的是页面浏览量或者点击量，页面被刷新一次就计算一次。
* UV(独立访客)：即Unique Visitor,访问您网站的一台电脑客户端为一个访客。

In [ ]:
# pv_daily记录每天用户操作次数
pv_daily=data_user.groupby('date').count()['user_id'].rename(columns={'user_id':'pv'})

In [ ]:
pv_daily.head(5)

In [ ]:
# uv_daily记录每天不同的上线用户数量
uv_daily  =  data_user.groupby('date')['user_id'].apply(lambda x:x.drop_duplicates().count())

In [ ]:
uv_daily  =  uv_daily.rename(columns={'user_id':'uv'})

In [ ]:
#将2个数据进行合并
pv_uv_daily = pd.concat([pv_daily,uv_daily],axis=1)

In [ ]:
pv_uv_daily.head(5)

In [ ]:
corr = pv_uv_daily.corr(method = 'spearman')#改为pearson
print( '访问量和访问用户数的相关系数为: %.2f' %corr.iloc[0,1])

In [ ]:
plt.figure(figsize=(16,9))
plt.subplot(211)
plt.plot(pv_daily,color = 'red')
plt.title('访问量变化趋势')
plt.subplot(212)
plt.plot(uv_daily,color = 'green')
plt.title('不同用户数量变化趋势')
plt.suptitle('PV和UV趋势',size=20)
plt.show()

* 通过图形观察，发现在双十二期间，pv和uv访问量达到峰值，并且可以发现，uv和pv两个访问量数值差距比较大

* 小时访问量分析

In [ ]:
# pv_daily记录每个小时用户操作次数
pv_daily  = data_user.groupby('hour').count()['user_id'].rename(columns={'user_id':'pv'})

In [ ]:
pv_daily.head(5)

In [ ]:
# uv_daily记录每个小时的上线用户数量
uv_daily  =  data_user.groupby('hour')['user_id'].apply(lambda x:x.drop_duplicates().count())

In [ ]:
uv_daily  =  uv_daily.rename(columns={'user_id':'uv'})

In [ ]:
#将2个数据进行合并
pv_uv_daily = pd.concat([pv_daily,uv_daily],axis=1)

In [ ]:
pv_uv_daily.rename(columns ={0:'pv',1:'uv'},inplace = True)

In [ ]:
pv_uv_daily.head(5)

In [ ]:
corr = pv_uv_daily.corr(method = 'spearman')#改为pearson
print( '访问量和访问用户数的相关系数为: %.2f' %corr.iloc[0,1])

In [ ]:
plt.figure(figsize=(16,8))
pv_uv_daily['pv'].plot( color = 'steelblue', label = '每个小时访问量')
plt.legend(loc='upper left')
plt.ylabel('借款笔数分布',labelpad =20)
pv_uv_daily['uv'].plot( color = 'red', secondary_y=True,label = '每个小时不同用户数量')
plt.ylabel('逾期占比情况',labelpad =20)
plt.xticks(range(0,24),pv_uv_daily.index)
plt.legend(loc='upper center')
plt.grid(True)
plt.show()

* pv和uv在凌晨0-5点期间波动情况相同，都呈下降趋势，访问量都比较小，同时在晚上18：00左右，pv波动情况比较剧烈，相比来看uv不太明显，因此晚上18：00以后是淘宝用户访问app的活跃时间段。

* 不同行为类型用户pv分析

In [ ]:
# 计算不同的用户行为，在每个小时的访问量
pv_detail = pd.pivot_table(columns ='behavior_type',index ='hour',data = data_user, values = 'user_id',
                           aggfunc = np.size)

In [ ]:
# 发现用户行为为点击的用户访问量最高
plt.figure(figsize=(12,8))
sns.lineplot(data=pv_detail)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.lineplot(data=pv_detail.iloc[:,1:])
plt.show()

* 通过图发现,点击这一用户行为相比较于其他三类用户行为，pv访问量较高，同时四种用户行为的波动情况基本一致，因此晚上这一时间段不管哪一种用户行为，pv访问量都是最高的。
   可以看出，加入购物车这一用户行为的pv总量高于收藏的总量，因此在后续漏斗流失分析中，用户类型3应该在2之前分析。

### 4.2 用户消费行为分析

* （1）用户购买次数情况分析



In [ ]:
# 取出支付的用户
data_user_buy = data_user[data_user.behavior_type=='4'].groupby('user_id').size()
data_user_buy.head(5)

In [ ]:
data_user_buy.describe()

In [ ]:
plt.hist(x =data_user_buy,bins=100 )
plt.show()

* 图表显示：淘宝用户消费次数普遍在10次以内，因此需要重点关注购买次数在10次以上的消费者用户群体。

* （2）分析日ARPPU
* ARPPU（average revenue per paying user）是指从每位付费用户身上获得的收入，它反映的是每个付费用户的平均付费额度
* ARPPU=总收入/活跃用户付费数量
* 本数据集中没有消费金额，因此在计算过程中用消费次数代替消费金额

In [ ]:
# 人均消费次数=消费总次数/消费人数
data_use_buy1=data_user[data_user.behavior_type=='4'].groupby(['date','user_id'])['behavior_type'].count().reset_index().rename(columns={'behavior_type':'total'})

In [ ]:
data_use_buy1

In [ ]:
# 统计每天的消费总次数和消费人数
data_use_buy2 = pd.DataFrame(data_use_buy1.groupby('date').sum()['total']/data_use_buy1.groupby('date').count()['total'])

In [ ]:
data_use_buy2.describe()

In [ ]:
data_use_buy2.plot()
plt.show()

* 平均每天消费次数在1-2次之间波动，双十二期间消费次数达到最高值。

* （3）日ARPU情况
* ARPU(Average Revenue Per User) ：平均每用户收入，可通过 总收入/AU 计算得出。它可以衡量产品的盈利能力和发展活力。
* 活跃用户数平均消费次数=消费总次数/活跃用户人数(每天有操作行为的为活跃)

In [ ]:
data_user['operation'] = 1
data_use_buy2=data_user.groupby(['date','user_id','behavior_type'])['operation'].count().reset_index().rename(columns={'operation':'total'})

In [ ]:
data_use_buy2.head(5)

In [ ]:
data_use_buy2.groupby('date').apply(lambda x:x[x.behavior_type=='4'].total.sum()/len(x.user_id.unique())).plot()
plt.title('daily_ARPU')
plt.show()

* 淘宝活跃用户（有过操作行为的用户）平均每天消费次数比较低，在0.58次左右，双十二期间达到最高值，有2左右。

* （4）付费率
* 付费率=消费人数/活跃用户人数


In [ ]:

data_use_buy2.groupby('date').apply(lambda x:x[x.behavior_type=='4'].total.count()/len(x.user_id.unique())).plot()
plt.title('daily_afford_rate')
plt.show()

* 每天活跃用户人群中，大概24%的用户具有消费行为，在双十二期间消费用户人数最多。

（5）同一时间段用户消费次数分布

In [ ]:
data_user_buy3 = data_user[data_user.behavior_type=='4'].groupby(['user_id','date','hour'])['operation'].sum().rename('buy_count')

In [ ]:
data_user_buy3.head(6)

In [ ]:
sns.distplot(data_user_buy3)
plt.show()

In [ ]:
print('大多数用户消费：{}次'.format(data_user_buy3.mode()[0]))

* 在同一时间段中，大多数用户消费次数为1次

### 4.3 复购情况分析

* 复购情况，即两天以上有购买行为,一天多次购买算一次
* 复购率=有复购行为的用户数/有购买行为的用户总数

In [ ]:
date_rebuy=data_user[data_user.behavior_type=='4'].groupby('user_id')['date'].apply(lambda x:len(x.unique())).rename('rebuy_count')

In [ ]:
date_rebuy #得到的是每个用户不同购买日期的次数

In [ ]:
print('复购率:',round(date_rebuy[date_rebuy>=2].count()/date_rebuy.count(),4))

In [ ]:
#所有复购时间间隔消费次数分布
# 计算不同的用户，不同的日期下的购买次数
data_day_buy=data_user[data_user.behavior_type=='4'].groupby(['user_id','date']).operation.count().reset_index()

In [ ]:
data_day_buy.head(5)

In [ ]:
data_user_buy4 = data_day_buy.groupby('user_id').date.apply(lambda x:x.sort_values().diff(1).dropna())

In [ ]:
data_user_buy4= data_user_buy4.map(lambda x:x.days)

In [ ]:
data_user_buy4.head(5)#输出不同的用户每次购物的日期间隔

In [ ]:
data_user_buy4.describe()

In [ ]:
data_user_buy4.value_counts().plot(kind='bar')
plt.title('time_gap')
plt.xlabel('gap_day')
plt.ylabel('gap_count')

* 多数用户复购率为0.4693，消费次数随着消费时间间隔的增加而不断下降，在1-10天之内复购次数比较多，10天之后复购次数淘宝用户很少在进行复购，因此需要重视10天之内的淘宝用户复购行为，增加用户复购。

In [ ]:
#不同用户平均复购时间分析
sns.distplot(data_user_buy4.reset_index().groupby('user_id').date.mean())
plt.show()

In [ ]:
data_user_buy4.quantile(np.linspace(0,1,11))#计算分位数


* 不同用户平均复购时间接近正态分布，但是总体来看，呈现逐渐下降趋势。多数淘宝用户平均复购时间集中在1-6天时间间隔内。

### 4.4 漏斗流失分析

In [ ]:
data_user_count=data_user.groupby(['behavior_type']).size()
data_user_count.head()

In [ ]:
pv_all = data_user['user_id'].count()
print(pv_all)

In [ ]:
print( '总浏览量—点击量 流失率等于%.2f%%' %   ( ( pv_all  - data_user_count[0])*100 / pv_all))    

In [ ]:
print( '点击量- 加入购物车量 流失率等于%.2f%%' %   ( ( data_user_count[0]  - data_user_count[2])*100 / data_user_count[0]))    

In [ ]:
print( '加购物车量-收藏量等于%.2f%%' %   ( ( data_user_count[2]  - data_user_count[1])*100 / data_user_count[2]))    

In [ ]:
print( '收藏量-购买量 流失率等于%.2f%%' %   ( ( data_user_count[1]  - data_user_count[3])*100 / data_user_count[1]))    

* 通过流失率情况的计算和分析，重视每一环节流失率可能发生的情况。

### 4.5 用户价值度RFM模型分析

RFM的含义：
* R（Recency）：客户最近一次交易时间的间隔。R值越大，表示客户交易发生的日期越久，反之则表示客户交易发生的日期越近。
* F（Frequency）：客户在最近一段时间内交易的次数。F值越大，表示客户交易越频繁，反之则表示客户交易不够活跃。
* M（Monetary）：客户在最近一段时间内交易的金额。M值越大，表示客户价值越高，反之则表示客户价值越低。

* RFM分析就是根据客户活跃程度和交易金额的贡献，进行客户价值细分的一种方法。

因为本数据集没有提供消费金额，因此只能R和F进行用户价值分析

In [ ]:
from datetime import datetime
datenow=datetime(2014,12,20)#参考时间

In [ ]:
#每位用户最近购买时间
recent_buy_time=data_user[data_user.behavior_type=='4'].groupby('user_id').date.apply(lambda x: datetime(2014,12,20)-x.sort_values().iloc[-1]).reset_index().rename(columns={'date':'recent'})
recent_buy_time.recent=recent_buy_time.recent.map(lambda x:x.days)

In [ ]:
#每个用户消费频率
buy_freq=data_user[data_user.behavior_type=='4'].groupby('user_id').date.count().reset_index().rename(columns={'date':'freq'})
rfm=pd.merge(recent_buy_time,buy_freq,left_on='user_id',right_on='user_id',how='outer')

In [ ]:
#将各维度分成两个程度,分数越高越好,基于等频分段
rfm['recent_value']=pd.qcut(rfm.recent,2,labels=['2','1'])
rfm['freq_value']=pd.qcut(rfm.freq,2,labels=['1','2'])
rfm['rfm']=rfm['recent_value'].str.cat(rfm['freq_value'])

In [ ]:
rfm.head()

## 五 结果解读


* 在上面每一次分析中，已经有了结论，这里不再重复描述
* 通过RF用户价值分析，对于22用户，为重点用户需要关注；对于21这类忠诚度高而购买能力不足的，可以可以适当给点折扣或捆绑销售来增加用户的购买频率。对于12这类忠诚度不高而购买能力强的，需要关注他们的购物习性做精准化营销。